In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('Arrest_Data_from_2010_to_Present.csv') # read in the csv file

In [ ]:
df.head()

In [ ]:
#delete columns not relevant to analysis
df.drop(['Report ID','Area ID','Charge Group Code','Location'],axis=1,inplace=True)

In [ ]:
print (df.info())

In [ ]:
#need to cleanup the time field...it is stored like 645 instead of 06:45
df_cleansed = df

#convert float to string
df_cleansed['Time'] = df_cleansed['Time'].astype(str) 

#get rid of decimals
df_cleansed['Time'] = df_cleansed['Time'].str.split(".", expand=True)[0] 

#convert missing to 0000
df_cleansed['Time'] = df_cleansed['Time'].replace(to_replace="nan",value="0000") 

#treat 0 as missing and convert to 0000
df_cleansed['Time'] = df_cleansed['Time'].replace(to_replace="0",value="0000") 

#2400 is not a valid time, converting to 0001 so it isn't the same as missing
df_cleansed['Time'] = df_cleansed['Time'].replace(to_replace="2400",value="0001") 

#split the time string to get the appropriate digits that correspond to hours and minutes
df_cleansed['Hour'] = np.where(df_cleansed['Time'].str.len() == 4,df_cleansed['Time'].str[-4:2],np.where(df_cleansed['Time'].str.len() == 3,df_cleansed['Time'].str[-3:1],"00"))
df_cleansed['Minute'] = df_cleansed['Time'].str[-2:4]

#put hour and minute back together in time format
df_cleansed['NewTime'] = pd.to_datetime(df_cleansed['Hour'] + ':' + df_cleansed['Minute'] + ':00',format='%H:%M:%S').dt.time

In [ ]:
#need to clean up cross street field

#remove duplicate whitespaces
df_cleansed['Cross Street'] = df_cleansed['Cross Street'].replace('\s+',' ',regex=True)
df_cleansed['Address'] = df_cleansed['Address'].replace('\s+',' ',regex=True)

#if all digits are numeric, nullify
df_cleansed['Address New'] = np.where(df_cleansed["Address"].str.isdigit() == True,np.nan, df_cleansed["Address"])
df_cleansed['Cross Street New'] = np.where(df_cleansed["Cross Street"].str.isdigit() == True,np.nan, df_cleansed["Cross Street"])

df_cleansed['Address_first_word'] = df_cleansed['Address'].str.split(n=1).str[0]
df_cleansed['Street'] = np.where(df_cleansed['Address_first_word'].str.isdigit() == True,df_cleansed['Address'].str.split(n=1).str[1],df_cleansed['Address'])

df_cleansed['Cross_street_first_word'] = df_cleansed['Cross Street'].str.split(n=1).str[0]
df_cleansed['CrossStreet'] = np.where(df_cleansed['Cross_street_first_word'].str.isdigit() == True,df_cleansed['Cross Street'].str.split(n=1).str[1],df_cleansed['Cross Street'])

In [ ]:
#delete columns not relevant to analysis
df_cleansed.drop(['Time','Hour','Minute','Address','Cross Street','Address New','Cross Street New','Address_first_word','Cross_street_first_word'],axis=1,inplace=True)

#add year column
df_cleansed['Date'] = pd.to_datetime(df_cleansed['Arrest Date'])
df_cleansed['Year'] = df_cleansed['Date'].dt.year
df_cleansed.head()

In [ ]:
#df_test = df_cleansed[df_cleansed['Address_numeric'] == True]
#print (df_test)
#df_test = df_cleansed.groupby(by='Cross Street')
#print (df_test['Cross Street'].count())
#df_test = df_cleansed.groupby(by='Reporting District')

In [ ]:
district_counts = df_cleansed['Reporting District'].value_counts().reset_index().rename(columns={'index': 'Key'})
district_counts.columns = ['Key', 'District Counts']
#district_counts

male = df_cleansed[df_cleansed['Sex Code'] == 'M']
male_counts = male['Reporting District'].value_counts().reset_index().rename(columns={'index': 'key'})
male_counts.columns = ['Key','Male_Counts']
#male_counts

female = df_cleansed[df_cleansed['Sex Code'] == 'F']
female_counts = female['Reporting District'].value_counts().reset_index().rename(columns={'index': 'key'})
female_counts.columns = ['Key','Female_Counts']
#female_counts

year_counts = df_cleansed.groupby(['Reporting District','Year']).size().reset_index()
year_counts.columns = ['Key','Year','Year_Dist_Counts']
#year_counts

merged1 = district_counts.merge(male_counts, left_on='Key', right_on='Key')
merged2 = merged1.merge(female_counts,left_on='Key', right_on='Key')
merged2.head()

In [ ]:
#year_counts.to_csv(r'counts.csv')
df_cleansed.to_csv(r'Cleansed.csv')

In [ ]:
#http://www.acgeospatial.co.uk/geopandas-shapefiles-jupyter/
#http://geohub.lacity.org/datasets/4398360b1a0242b78904f46b3786ae73_0/data

import geopandas as gpd
gdf = gpd.read_file('LAPD_Reporting_Districts.shp')
#print (gdf)

#merge polygon dataframe with counts by reporting district
gdf1 = gdf.merge(merged2, left_on='REPDIST', right_on='Key')
gdf1['Counts by Area'] = gdf1['District Counts']/gdf1['AREA']
gdf2 = gdf1.merge(year_counts, left_on='REPDIST',right_on='Key')
print(gdf2.info())

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline
#gdf3.head()
gdf1.plot(column='District Counts', cmap=None, figsize=(10, 10),  legend=True)
plt.title('Arrests by District')
gdf1.plot(column='Counts by Area', cmap=None, figsize=(10, 10),  legend=True)
plt.title('Arrests by Distict / Area')
gdf1.plot(column='Male_Counts', cmap=None, figsize=(10, 10),  legend=True)
plt.title('Male Arrests by Distict')
gdf1.plot(column='Female_Counts', cmap=None, figsize=(10, 10),  legend=True)
plt.title('Female Arrests by Distict')

In [ ]:
#https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0
#https://github.com/CrazyDaffodils/Interactive-Choropleth-Map-Using-Python

import json

#Read data to json
merged_json = json.loads(gdf2.to_json())

#Convert to str like object
json_data = json.dumps(merged_json)


In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][9]

#Reverse color order so that dark blue is highest obesity.
#palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 2500)

#Define custom tick labels for color bar.
#tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
#border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
border_line_color=None,location = (0,0), orientation = 'horizontal')

#Create figure object.
p = figure(title = 'Los Angeles Arrests by Reporting District', plot_height = 950 , plot_width = 600, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'District Counts', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(p)

In [ ]:
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

#Define function that returns json_data for year selected by user.
    
def json_data(selectedYear):
    yr = selectedYear
    df_yr = gdf2[gdf2['Year'] == yr]
    #merged = gdf.merge(df_yr, left_on = 'country_code', right_on = 'code', how = 'left')
    #merged.fillna('No data', inplace = True)
    merged_json = json.loads(df_yr.to_json())
    json_data = json.dumps(merged_json)
    return json_data

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data(2010))

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][9]

#Reverse color order so that dark blue is highest obesity.
#palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 2500)

#Define custom tick labels for color bar.
#tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Add hover tool
hover = HoverTool(tooltips = [ ('Reporting District','@REPDIST'),('Arrests', '@Year_Dist_Counts')])


#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal')


#Create figure object.
p = figure(title = 'Los Angeles Arrests by Reporting District', plot_height = 850 , plot_width = 600, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Year_Dist_Counts', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)


p.add_layout(color_bar, 'below')

# Define the callback function: update_plot
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    p.title.text = 'Arrest Counts' %yr
    
# Make a slider object: slider 
slider = Slider(title = 'Year',start = 2010, end = 2019, step = 1, value = 2010)
slider.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(layout)